In [215]:
#ouvre le xlsx Dataset 

import pandas as pd

#ouvre le xlsx Dataset
df = pd.read_excel('Dataset.xlsx')
# Supposons que votre DataFrame s'appelle df
btc_columns = df.filter(like='BTC').columns.tolist() + df.filter(like='btc').columns.tolist()
btc_columns = df.filter(like='ETH').columns.tolist() + df.filter(like='eth').columns.tolist()
#sors moi un df avec les colonnes btc, en ajoutant la colonne Date
df_btc = df[btc_columns + ['date']]
#sur la colonne Close, fait un pct change dans une nouvelle colonne Return
df_btc['Return_BTC'] = df_btc['Close_ETH'].pct_change()
#drop la premiere ligne de df_btc
df_btc = df_btc.drop(df_btc.index[0])
# Supposons que 'value' est la colonne contenant les valeurs que vous voulez comparer
df_btc['target'] = (df_btc['Return_BTC'].shift(-1) > df_btc['Return_BTC']).astype(int)
# Supposons que 'date' est la colonne contenant les dates
df_btc['date'] = pd.to_datetime(df_btc['date'])  # Assurez-vous que la colonne est de type datetime
df_btc['year'] = df_btc['date'].dt.year
df_btc['month'] = df_btc['date'].dt.month
df_btc['day'] = df_btc['date'].dt.day
df_btc['day_of_week'] = df_btc['date'].dt.dayofweek  # Lundi=0, Dimanche=6

#fais un fillna 0
df_btc = df_btc.fillna(0)
#fais le train de 2017-08 a 2022-08 et le reste en test
df_train = df_btc[df_btc['date'] < '2022-08-01']
df_test = df_btc[df_btc['date'] >= '2022-08-01']
##mets la colonne date en index 
df_train = df_train.set_index('date')
df_test = df_test.set_index('date')

X_train = df_train.drop('target',axis=1)
y_train = df_train['target']
X_test = df_test.drop('target',axis=1)
y_test = df_test['target']


/var/folders/bn/74x_l6t57mncrtd65lq_drbw0000gn/T/ipykernel_30323/898354688.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_btc['Return_BTC'] = df_btc['Close_ETH'].pct_change()


In [216]:
from sklearn.preprocessing import StandardScaler

df_btc_scaled = df_btc.copy()

# Obtenir toutes les colonnes sauf la colonne de date
columns_to_scale = df_btc.columns.drop(['date', 'target','day','month','year','day_of_week'])

scaler = StandardScaler()
df_btc_scaled[columns_to_scale] = scaler.fit_transform(df_btc[columns_to_scale])

df_btc_scaled.head()


from sklearn.preprocessing import MinMaxScaler

df_btc_scaled_mm = df_btc.copy()

mmscaler = MinMaxScaler()
df_btc_scaled_mm[columns_to_scale] = mmscaler.fit_transform(df_btc[columns_to_scale])

df_btc_scaled_mm.head()

,Close_ETH,Volume_ETH,eth_tweet_count,eth_posts_count,eth_textblob_polarity_min,eth_textblob_polarity_max,eth_textblob_polarity_mean,eth_vader_polarity_compound_min,eth_vader_polarity_compound_max,eth_vader_polarity_compound_mean,date,Return_BTC,target,year,month,day,day_of_week
1,0.044482,0.001818,0.004474,0.682927,0.835544,0.50,0.299749,0.974778,0.621018,0.271410,2017-08-18,0.479407,1,2017,8,18,4
2,0.050690,0.000892,0.006067,0.512195,0.464191,0.50,0.196069,0.363965,0.841715,0.277640,2017-08-21,0.623908,0,2017,8,21,0
3,0.047835,0.001479,0.007811,0.707317,0.657604,0.25,0.066698,0.444618,0.600481,0.316604,2017-08-22,0.462139,1,2017,8,22,1
4,0.049092,0.001631,0.005081,0.512195,0.851017,0.70,0.319808,0.491935,0.867361,0.442096,2017-08-23,0.531762,1,2017,8,23,2
5,0.050718,0.001221,0.004702,0.707317,0.696286,0.60,0.206135,0.415681,0.613504,0.386197,2017-08-24,0.537650,0,2017,8,24,3


In [217]:

from sklearn.ensemble import RandomForestClassifier

def train_random_forest(X_train, y_train, X_test):
    
    # Créer le modèle de forêt aléatoire
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Entraîner le modèle
    rf.fit(X_train, y_train)

    # Prédire les valeurs de l'ensemble de test
    y_pred = rf.predict(X_test)

    # Afficher la précision du modèle
    from sklearn.metrics import accuracy_score
    print('Accuracy:', accuracy_score(y_test, y_pred))
    #montre le RMSE 
    from sklearn.metrics import mean_squared_error
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

    #compte le nombre de 0 de df 
    print(X_test.value_counts())
    # Retourner le modèle entraîné et les prédictions
    return rf, y_pred

# Utiliser la fonction
rf_model, rf_pred = train_random_forest(X_train, y_train, X_test)



Accuracy: 0.6529411764705882
RMSE: 0.5891169862849074
Close_ETH  Volume_ETH    eth_tweet_count  eth_posts_count  eth_textblob_polarity_min  eth_textblob_polarity_max  eth_textblob_polarity_mean  eth_vader_polarity_compound_min  eth_vader_polarity_compound_max  eth_vader_polarity_compound_mean  Return_BTC  year  month  day  day_of_week
1102.73    2.494798e+06  575.0            30.0             -0.221212                  0.600000                   0.113642                    -0.8442                          0.8807                           0.033787                          -0.173843   2022  11     9    2              1
1609.01    1.419827e+06  548.0            29.0             -0.400000                  0.700000                   0.116384                    -0.6258                          0.9744                           0.262541                          -0.128564   2022  8      19   4              1
1578.48    5.275393e+05  526.0            0.0               0.000000                  0

In [218]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
import matplotlib.pyplot as plt

def logistic_regression(X_train, y_train, X_test, y_test):
    
    # Logistic Regression
    logreg = LogisticRegression(random_state=42)
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)

    # Evaluation: Confusion matrix
    logreg_acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred) # Confusion matrix
    tpr_logreg = cm[1][1] /(cm[1][0] + cm[1][1])

    print('The accuracy score is:', logreg_acc) # accuracy score
    print('Sensitivity (TPR) =', tpr_logreg)

    print('\n Confusion matrix \n \n')
    print(classification_report(y_test, y_pred))

    # Calculate the RMSE 
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
    return y_pred


logpred = logistic_regression(X_train, y_train, X_test, y_test)

The accuracy score is: 0.5411764705882353
Sensitivity (TPR) = 0.6705882352941176

 Confusion matrix 
 

              precision    recall  f1-score   support

           0       0.56      0.41      0.47        85
           1       0.53      0.67      0.59        85

    accuracy                           0.54       170
   macro avg       0.54      0.54      0.53       170
weighted avg       0.54      0.54      0.53       170

RMSE: 0.6773651374345779


In [219]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
import numpy as np

def linear_discriminant_analysis(X_train, y_train, X_test, y_test):
    lda = LinearDiscriminantAnalysis(solver='lsqr', store_covariance=True)
    lda.fit(X_train, y_train)

    # Predict Test Set Responses
    y_predicted = lda.predict(X_test)
    y_predicted= np.array(y_predicted > 0.5, dtype=float)

    # Evaluation: Confusion matrix
    lda_acc = accuracy_score(y_test, y_predicted)  # accuracy score
    cm_lda = confusion_matrix(y_test, y_predicted) # Confusion matrix 
    tpr_lda = cm_lda[1][1] /(cm_lda[1][0] + cm_lda[1][1])

    print('Accuracy =', lda_acc)  
    print('Sensitivity (TPR) =', tpr_lda)

    print('\n Confusion matrix \n \n')
    print(classification_report(y_test, y_predicted ))

    # Calculate the RMSE
    print('RMSE:', mean_squared_error(y_test, y_predicted, squared=False))
    return y_predicted

#lance la fonction
discri_pred = linear_discriminant_analysis(X_train, y_train, X_test, y_test)

Accuracy = 0.711764705882353
Sensitivity (TPR) = 0.6235294117647059

 Confusion matrix 
 

              precision    recall  f1-score   support

           0       0.68      0.80      0.74        85
           1       0.76      0.62      0.68        85

    accuracy                           0.71       170
   macro avg       0.72      0.71      0.71       170
weighted avg       0.72      0.71      0.71       170

RMSE: 0.5368754921931593


In [220]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error

def decision_tree_classifier(X_train, y_train, X_test, y_test):
    dtree = DecisionTreeClassifier(random_state=42)

    # Build classification tree
    dtree.fit(X_train, y_train)

    y_pred = dtree.predict(X_test)

    # Evaluation: Confusion matrix
    dtree_acc = accuracy_score(y_test, y_pred)   # accuracy score
    cm_dtree = confusion_matrix(y_test, y_pred) # Confusion matrix 
    tpr_dtree = cm_dtree[1][1] /(cm_dtree[1][0] + cm_dtree[1][1])

    print("Accuracy:",dtree_acc)
    print('Sensitivity (TPR) =', tpr_dtree)
    print('\n Confusion matrix \n \n')
    print(classification_report(y_test, y_pred ))

    # Calculate the RMSE
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
    return y_pred

decision_pred = decision_tree_classifier(X_train, y_train, X_test, y_test)

Accuracy: 0.6235294117647059
Sensitivity (TPR) = 0.611764705882353

 Confusion matrix 
 

              precision    recall  f1-score   support

           0       0.62      0.64      0.63        85
           1       0.63      0.61      0.62        85

    accuracy                           0.62       170
   macro avg       0.62      0.62      0.62       170
weighted avg       0.62      0.62      0.62       170

RMSE: 0.6135719910778964


In [221]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error

def gradient_boosting_classifier(X_train, y_train, X_test, y_test):
    booster = GradientBoostingClassifier(max_depth=7,n_estimators=50,min_samples_split=1400,min_samples_leaf=60,max_features=7,subsample=0.85)
    boost_est = booster.fit(X_train, y_train)

    y_pred = boost_est.predict(X_test)

    # Evaluation: Confusion matrix
    boosting_acc = accuracy_score(y_test, y_pred) #accuracy score
    cm_bossting = confusion_matrix(y_test, y_pred) # Confusion matrix 
    tpr_boost = cm_bossting[1][1] /(cm_bossting[1][0] + cm_bossting[1][1]) #Sensitivity (TPR)

    print('Accuracy:', boosting_acc) # accuracy score
    print('Sensitivity (TPR) =', tpr_boost)

    print('\n Confusion matrix \n \n')
    print(classification_report(y_test, y_pred ))

    # Calculate the RMSE
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))
    return y_pred

gradient_pred = gradient_boosting_classifier(X_train, y_train, X_test, y_test)

Accuracy: 0.5
Sensitivity (TPR) = 0.0

 Confusion matrix 
 

              precision    recall  f1-score   support

           0       0.50      1.00      0.67        85
           1       0.00      0.00      0.00        85

    accuracy                           0.50       170
   macro avg       0.25      0.50      0.33       170
weighted avg       0.25      0.50      0.33       170

RMSE: 0.7071067811865476


/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [222]:
# input les données normaliser patati patata

from sklearn.neighbors import KNeighborsClassifier
# Creating classifiers for every value of K
classifiers = []
nb_neighbor = [1, 5, 10, 20, 30, 40, 50]
for i in range(len(nb_neighbor)):
    classifiers.append(KNeighborsClassifier(nb_neighbor[i]))

# Initializing the lists for accuracy, true positive rate and true negative rate
# Later used to compare the classifiers for different values of K
score_list = []
true_positive = []
true_negative = []

# Fitting the training dataset for every classifier and calculating metrics

index = 0
for clf in classifiers: 
    clf.fit(X_train,y_train)

    score = clf.score(X_test, y_test)  
    y_pred = clf.predict(X_test)
    
    print(f"Accuracy for K =", nb_neighbor[index] ,"nearest Neighbors: ",  accuracy_score(y_test, y_pred))
    # print le rmse
    from sklearn.metrics import mean_squared_error
    print('RMSE:', mean_squared_error(y_test, y_pred, squared=False))

    
    cm = confusion_matrix(y_test, y_pred) # Confusion matrix  

    score_list.append(score)
    true_positive.append(cm[1][1])
    true_negative.append(cm[0][0])
    
    index = index + 1

Accuracy for K = 1 nearest Neighbors:  0.4294117647058823
RMSE: 0.7553729114113886
Accuracy for K = 5 nearest Neighbors:  0.47058823529411764
RMSE: 0.7276068751089989
Accuracy for K = 10 nearest Neighbors:  0.5352941176470588
RMSE: 0.6816933932149711
Accuracy for K = 20 nearest Neighbors:  0.5764705882352941
RMSE: 0.6507913734559685
Accuracy for K = 30 nearest Neighbors:  0.5823529411764706
RMSE: 0.6462561866810479
Accuracy for K = 40 nearest Neighbors:  0.5588235294117647
RMSE: 0.6642111641550714
Accuracy for K = 50 nearest Neighbors:  0.5
RMSE: 0.7071067811865476


In [223]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
import numpy as np

def quadratic_discriminant_analysis(X_train, y_train, X_test, y_test):
    qdaClassifier = QuadraticDiscriminantAnalysis(store_covariance=True)
    qdaClassifier.fit(X_train,y_train)

    # Get predictions
    y_predict = qdaClassifier.predict(X_test)
    y_predicted= np.array(y_predict > 0.5, dtype=float)

    # Get evaluation criteria
    qda_acc = accuracy_score(y_test, y_predicted) 
    qda_cm = confusion_matrix(y_test, y_predicted)
    qda_tpr = qda_cm[1][1] /(qda_cm[1][0] + qda_cm[1][1])

    print('Accuracy =', qda_acc)
    print('Sensitivity (TPR) =', qda_tpr)

    print('\n Confusion matrix \n \n')
    print(classification_report(y_test, y_predicted ))

    # Calculate the RMSE
    print('RMSE:', mean_squared_error(y_test, y_predicted, squared=False))
    return y_predicted

quadratic_pred = quadratic_discriminant_analysis(X_train, y_train, X_test, y_test)

Accuracy = 0.5352941176470588
Sensitivity (TPR) = 0.6823529411764706

 Confusion matrix 
 

              precision    recall  f1-score   support

           0       0.55      0.39      0.46        85
           1       0.53      0.68      0.59        85

    accuracy                           0.54       170
   macro avg       0.54      0.54      0.53       170
weighted avg       0.54      0.54      0.53       170

RMSE: 0.6816933932149711


In [224]:
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
import xgboost as xgb

def xgboost_classifier(X_train, y_train, X_test, y_test, df_test):
    dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
    dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)
    dpred_clf = xgb.DMatrix(df_test, enable_categorical=True)

    params = {"objective": "multi:softmax","num_class": 3, "tree_method": "hist",
                "learning_rate": 0.3, "max_depth": 6,
                "gamma": 0, "subsample": 1, "colsample_bytree": 1,
                "alpha": 0, "lambda": 1,"random_state": 0}

    n = 50
    evals = [(dtest_clf, "validation"), (dtrain_clf, "train")]

    model = xgb.train(
       params=params,
       dtrain=dtrain_clf,
       num_boost_round=n,
       evals=evals,
       verbose_eval=1,
       # Activate early stopping
       early_stopping_rounds=30
    )
    preds = model.predict(dtest_clf)
    rmse = mean_squared_error(y_test, preds, squared=False)
    accuracy = accuracy_score(y_test, preds.round())

    print(f"RMSE of the base model: {rmse:.3f}")
    print(f"Accuracy of the base model: {accuracy:.3f}") 

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(y_test, preds)
    print("Confusion matrix: \n", conf_matrix)
    return preds

xgb_pred = xgboost_classifier(X_train, y_train, X_test, y_test, df_test)

[0]	validation-mlogloss:0.93839	train-mlogloss:0.85431
[1]	validation-mlogloss:0.83887	train-mlogloss:0.69566
[2]	validation-mlogloss:0.78052	train-mlogloss:0.58871
[3]	validation-mlogloss:0.73040	train-mlogloss:0.51224
[4]	validation-mlogloss:0.69677	train-mlogloss:0.44926
[5]	validation-mlogloss:0.67627	train-mlogloss:0.40370
[6]	validation-mlogloss:0.65769	train-mlogloss:0.36569
[7]	validation-mlogloss:0.64938	train-mlogloss:0.33827
[8]	validation-mlogloss:0.63924	train-mlogloss:0.31477
[9]	validation-mlogloss:0.62754	train-mlogloss:0.28965
[10]	validation-mlogloss:0.63422	train-mlogloss:0.27107
[11]	validation-mlogloss:0.63185	train-mlogloss:0.25724
[12]	validation-mlogloss:0.62668	train-mlogloss:0.24480
[13]	validation-mlogloss:0.62338	train-mlogloss:0.23053
[14]	validation-mlogloss:0.62281	train-mlogloss:0.21843
[15]	validation-mlogloss:0.61891	train-mlogloss:0.20838
[16]	validation-mlogloss:0.62092	train-mlogloss:0.20037
[17]	validation-mlogloss:0.63566	train-mlogloss:0.19075
[1

In [225]:
import pandas as pd

# Supposons que preds_dtree, preds_boost, preds_qda et preds_xgb sont vos prédictions
df_preds = pd.DataFrame({
    'RF': rf_pred,
    'logreg': logpred,
    'discri': discri_pred,
    'dtree': decision_pred,
    'boost': gradient_pred,
    'qda': quadratic_pred,
    'xgb': xgb_pred
}, index=X_test.index)

In [226]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
import time

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [227]:
# #Machine Learning Algorithm (MLA) Selection and Initialization
# MLA = [
#         #Ensemble Methods
#         ensemble.AdaBoostClassifier(),
#         ensemble.BaggingClassifier(),
#         ensemble.ExtraTreesClassifier(),
#         ensemble.GradientBoostingClassifier(),
#         ensemble.RandomForestClassifier(),

#         #Gaussian Processes
#         gaussian_process.GaussianProcessClassifier(),
        
#         #GLM
#         linear_model.LogisticRegressionCV(),
#         linear_model.PassiveAggressiveClassifier(),
#         linear_model.RidgeClassifierCV(),
#         linear_model.SGDClassifier(),
#         linear_model.Perceptron(),
        
#         #Navies Bayes
#         naive_bayes.BernoulliNB(),
#         naive_bayes.GaussianNB(),
        
#         #Nearest Neighbor
#         neighbors.KNeighborsClassifier(),
        
#         #SVM
#         svm.SVC(probability=True),
        
#         #Trees    
#         tree.DecisionTreeClassifier(),
#         tree.ExtraTreeClassifier(),
        
#         #Discriminant Analysis
#         discriminant_analysis.LinearDiscriminantAnalysis(),
#         discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
#         #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
#         XGBClassifier()    
#         ]



# #split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
# #note: this is an alternative to train_test_split
# #cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

# #create table to compare MLA metrics
# MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean','MLA Time','MLA RMSE']
# MLA_compare = pd.DataFrame(columns = MLA_columns)

# #create table to compare MLA predictions
# MLA_predict = y_train.copy()

# #index through MLA and save performance to table
# row_index = 0
# for alg in MLA:

#     #set name and parameters
#     MLA_name = alg.__class__.__name__
#     MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
#     MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
#     # #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
#     # cv_results = model_selection.cross_validate(alg, X_train, y_train, cv  = cv_split)

#     # #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
#     # MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

#     #save MLA predictions - see section 6 for usag
#     start_time = time.time()
#     alg.fit(X_train, y_train)
#     predictions = alg.predict(X_test)
#     end_time = time.time()
#     # Calculer le RMSE et l'ajouter à la table
#     accuracy = metrics.accuracy_score(y_test, predictions)
#     rmse = np.sqrt(mean_squared_error(y_test, predictions))
#     # Save the metrics to the table
#     MLA_compare.loc[row_index, 'MLA Time'] = end_time - start_time
#     MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = accuracy
#     MLA_compare.loc[row_index, 'MLA RMSE'] = rmse
    
    
#     MLA_predict[MLA_name] = predictions
    
#     row_index+=1

    
# #print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
# MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
# MLA_compare1 = MLA_compare.copy()
# MLA_compare

In [228]:
def run_ml_models(X_train, y_train, X_test, y_test,df):
    from sklearn import ensemble, gaussian_process, linear_model, naive_bayes, neighbors, svm, tree, discriminant_analysis
    from sklearn.metrics import accuracy_score, mean_squared_error
    from xgboost import XGBClassifier
    import pandas as pd
    import numpy as np
    import time

    seed = 42  # Vous pouvez choisir n'importe quel nombre pour la graine

    MLA = [
        ensemble.AdaBoostClassifier(random_state=seed),
        ensemble.BaggingClassifier(random_state=seed),
        ensemble.ExtraTreesClassifier(random_state=seed),
        ensemble.GradientBoostingClassifier(random_state=seed),
        ensemble.RandomForestClassifier(random_state=seed),
        gaussian_process.GaussianProcessClassifier(random_state=seed),
        linear_model.LogisticRegressionCV(random_state=seed),
        linear_model.PassiveAggressiveClassifier(random_state=seed),
        linear_model.RidgeClassifierCV(),  # Pas de random_state pour RidgeClassifierCV
        linear_model.SGDClassifier(random_state=seed),
        linear_model.Perceptron(random_state=seed),
        naive_bayes.BernoulliNB(),
        naive_bayes.GaussianNB(),
        neighbors.KNeighborsClassifier(),
        svm.SVC(probability=True, random_state=seed),
        tree.DecisionTreeClassifier(random_state=seed),
        tree.ExtraTreeClassifier(random_state=seed),
        discriminant_analysis.LinearDiscriminantAnalysis(),
        discriminant_analysis.QuadraticDiscriminantAnalysis(),
        XGBClassifier(random_state=seed)    
    ]

    MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean','MLA Time','MLA RMSE']
    MLA_compare = pd.DataFrame(columns = MLA_columns)
    MLA_predict = y_train.copy()

    row_index = 0
    for alg in MLA:
        MLA_name = alg.__class__.__name__
        MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
        MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

        start_time = time.time()
        alg.fit(X_train, y_train)
        predictions = alg.predict(X_test)
        alg.fit(df.drop('target', axis=1), df['target'])
        last_date_prediction = alg.predict(df.drop('target', axis=1).iloc[-1:])
        end_time = time.time()

        accuracy = accuracy_score(y_test, predictions)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))

        MLA_compare.loc[row_index, 'MLA Time'] = end_time - start_time
        MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = accuracy
        MLA_compare.loc[row_index, 'MLA RMSE'] = rmse

        MLA_predict[MLA_name] = last_date_prediction

        row_index+=1

    MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
    return MLA_compare, MLA_predict


In [229]:

# MLA_compare, MLA_predict = run_ml_models(X_train, y_train, X_test, y_test,df)

In [230]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Supposons que df_full est votre DataFrame complet qui contient toutes les données

# Fonction pour créer un DataFrame pour une période donnée
def create_df(start_date, end_date):
    # Pour cet exemple, nous allons simplement filtrer df_full pour obtenir les données dans la période spécifiée.
    df = df_btc[(df_btc['date'] >= start_date) & (df_btc['date'] <= end_date)]
    return df



def predict_next_day(X_train, y_train, X_test, y_test, df, current_month, previous_month, best_model):
    # Si nous sommes dans un nouveau mois ou si aucun meilleur modèle n'a encore été trouvé
    if best_model_name is None or previous_month != current_month:
        MLA_compare, MLA_predict = run_ml_models(X_train, y_train, X_test, y_test, df)
        
        # Calculer le ratio Accuracy/RMSE pour chaque modèle
        MLA_compare['Accuracy/RMSE'] = MLA_compare['MLA Test Accuracy Mean'] / MLA_compare['MLA RMSE']
        
        # Trouver le modèle avec le meilleur ratio Accuracy/RMSE
        MLA_compare['Accuracy/RMSE'] = pd.to_numeric(MLA_compare['Accuracy/RMSE'], errors='coerce')
        best_model = MLA_compare.loc[MLA_compare['Accuracy/RMSE'].idxmax()]
        print(best_model['MLA Name'])
        previous_month = current_month
    else :
        previous_month = current_month
        MLA_compare, MLA_predict = run_ml_models(X_train, y_train, X_test, y_test, df)
        best_model = best_model
        print(best_model['MLA Name'])
        previous_month = current_month

    
    
    return MLA_predict[best_model['MLA Name']], best_model['MLA Name'], best_model['MLA Test Accuracy Mean'], best_model['MLA RMSE'], previous_month, best_model


# Initialiser la date de début et la date de fin
prediction_start_date = datetime(2022, 8, 1)
#end_date = df_btc['date'].max()
end_date = datetime(2022,10,1)


#isole un df qui contient les données entre les deux dates 
df = create_df(prediction_start_date, end_date)

#récupère toute la colonne date afin de boucler dessus donc dans un nouveau df 
df_date = df['date']

#mets df_date dans un dataframe
df_date = pd.DataFrame(df_date)

# Initialiser une liste vide pour stocker les dates de début
start_dates = []

# Parcourir chaque date dans df_date
for date in df_date['date']:
    # Calculer la date de début comme étant six mois avant la date actuelle
    start_date = date - pd.DateOffset(months=6)
    
    # Vérifier si start_date est dans df_btc['date']
    if start_date not in df_btc['date'].values:
        # Si start_date n'est pas dans df_btc['date'], trouver la date la plus proche qui est dans df_btc['date']
        start_date = df_btc['date'][df_btc['date'].sub(start_date).abs().idxmin()]
    
    # Ajouter la date de début à la liste start_dates
    start_dates.append(start_date)

# Ajouter la liste start_dates comme une nouvelle colonne 'start_date' dans df
df_date['start_date'] = start_dates
# Initialiser le DataFrame pour stocker les prédictions
predictions_df = pd.DataFrame(columns=['date', 'prediction'])

# Initialiser le meilleur modèle
best_model_name = None
best_model_accuracy = None
best_model_rmse = None
current_month = None
previous_month = None
best_model = None

# Boucle sur chaque ligne dans df_Date
for index, row in df_date.iterrows():
    # Utiliser la valeur de la colonne 'date' comme current_date
    current_date = row['date']
    # Utiliser la valeur de la colonne 'start_date' comme window_start_date
    window_start_date = row['start_date']

    # Créer le DataFrame pour la période de la fenêtre glissante
    df = create_df(window_start_date, current_date)
    print(df['date'].min(), df['date'].max())

    # Supposons que df est votre DataFrame
    train_ratio = 0.8
    train_size = int(len(df) * train_ratio)

    df_train = df.iloc[:train_size]
    df_test = df.iloc[train_size:]

    df_train = df_train.set_index('date')
    df_test = df_test.set_index('date')

    X_train = df_train.drop('target',axis=1)
    y_train = df_train['target']
    X_test = df_test.drop('target',axis=1)
    y_test = df_test['target']

    current_month = df['month'].iloc[-1]

    #drop la colonne date
    df = df.drop('date', axis=1)
    
    # Prédire la valeur pour le jour suivant
    next_day_prediction, best_model_name, best_model_accuracy, best_model_rmse, previous_month, best_model = predict_next_day(X_train, y_train, X_test, y_test, df, current_month, previous_month, best_model)
    
    # Ajouter la prédiction, le nom du modèle, l'accuracy et le RMSE au DataFrame des prédictions
    predictions_df = predictions_df.append({
        'date': current_date, 
        'prediction': next_day_prediction,
        'model': best_model_name,
        'accuracy': best_model_accuracy,
        'RMSE': best_model_rmse
    }, ignore_index=True)

# Afficher les prédictions
print(predictions_df)

2022-02-01 00:00:00 2022-08-01 00:00:00


/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


XGBClassifier
2022-02-02 00:00:00 2022-08-02 00:00:00


/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


XGBClassifier
2022-02-03 00:00:00 2022-08-03 00:00:00


...

ExtraTreesClassifier
2022-03-29 00:00:00 2022-09-29 00:00:00


/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


ExtraTreesClassifier
2022-03-30 00:00:00 2022-09-30 00:00:00


/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/Users/VerTebr0/opt/anaconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


ExtraTreesClassifier
         date prediction      RMSE  accuracy                 model
0  2022-08-01        [1]  0.480384  0.769231         XGBClassifier
1  2022-08-02        [0]  0.480384  0.769231         XGBClassifier
2  2022-08-03        [1]  0.480384  0.769231         XGBClassifier
3  2022-08-04        [1]  0.480384  0.769231         XGBClassifier
4  2022-08-05        [0]  0.480384  0.769231         XGBClassifier
5  2022-08-08        [0]  0.480384  0.769231         XGBClassifier
6  2022-08-09        [1]  0.480384  0.769231         XGBClassifier
7  2022-08-10        [0]  0.480384  0.769231         XGBClassifier
8  2022-08-11        [1]  0.480384  0.769231         XGBClassifier
9  2022-08-12        [0]  0.480384  0.769231         XGBClassifier
10 2022-08-15        [1]  0.480384  0.769231         XGBClassifier
11 2022-08-16        [0]  0.480384  0.769231         XGBClassifier
12 2022-08-17        [1]  0.480384  0.769231         XGBClassifier
13 2022-08-18        [0]  0.480384  0.769

In [231]:
# Fusionner predictions_df et df_btc sur la colonne 'date'
df_merged = pd.merge(predictions_df, df_btc[['date', 'target']], on='date')

# Comparer les colonnes 'prediction' et 'target'
df_merged['correct'] = df_merged['prediction'] == df_merged['target']

# Compter le nombre de prédictions correctes
num_correct_predictions = df_merged['correct'].sum()

print(f"Nombre de prédictions correctes : {num_correct_predictions}")

Nombre de prédictions correctes : 45
